In [ ]:
import torch
import json
import time
import re
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from sentence_transformers import SentenceTransformer
!pip install hnswlib
!pip install -U bitsandbytes
import hnswlib

# Configuration
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
DATA_DIR = Path("./anton_data")
DATA_DIR.mkdir(exist_ok=True)

# Crisis Keywords for Safety Protocol
CRISIS_KEYWORDS = [
    "suicide", "kill myself", "end my life", "not worth living",
    "self-harm", "hurt myself", "cut myself", "want to die",
    "planning to die", "imminent harm", "goodbye forever",
    "better off dead", "no reason to live"
]

class AntonChatbot:
    """
    Anton - A warm, peer-like AI companion for new city settlers
    Guides users through 7-day onboarding while providing emotional support
    """

    def __init__(self, user_id: str, user_name: str = None, user_city: str = "Bangalore"):
        self.user_id = user_id
        self.user_name = user_name or "friend"
        self.user_city = user_city
        self.user_dir = DATA_DIR / user_id
        self.user_dir.mkdir(exist_ok=True)

        # Initialize Mistral model
        print("🤖 Loading Anton's AI brain (Mistral-7B)...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16
        )

        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.float16
        )
        self.model.eval()

        # Initialize embedding model for vector storage
        print("🧠 Loading memory system...")
        self.embedding_model = SentenceTransformer(EMBEDDING_MODEL)

        # Load or initialize user state
        self.state = self._load_state()
        self.onboarding_step = self.state.get('onboarding_step', 1)
        self.onboarding_complete = self.state.get('onboarding_complete', False)
        self.onboarding_data = self.state.get('onboarding_data', {})
        self.conversation_history = self.state.get('conversation_history', [])
        self.emotional_logs = self.state.get('emotional_logs', [])
        self.emergency_contact = self.state.get('emergency_contact', None)

        # Initialize HNSW index for context retrieval
        self.vector_dim = 384
        self.index = self._load_or_create_index()

        # Track current question state
        self.current_question = None
        self.waiting_for_response = False

    def _load_state(self) -> Dict:
        """Load user state from disk"""
        state_file = self.user_dir / "state.json"
        if state_file.exists():
            with open(state_file, 'r') as f:
                return json.load(f)
        return {}

    def _save_state(self):
        """Save user state to disk"""
        self.state['onboarding_step'] = self.onboarding_step
        self.state['onboarding_complete'] = self.onboarding_complete
        self.state['onboarding_data'] = self.onboarding_data
        self.state['conversation_history'] = self.conversation_history
        self.state['emotional_logs'] = self.emotional_logs
        self.state['emergency_contact'] = self.emergency_contact
        self.state['last_updated'] = datetime.now().isoformat()

        with open(self.user_dir / "state.json", 'w') as f:
            json.dump(self.state, f, indent=2)

    def _load_or_create_index(self) -> hnswlib.Index:
        """Load existing HNSW index or create new one"""
        index_file = self.user_dir / "memory_index.bin"
        index = hnswlib.Index(space='cosine', dim=self.vector_dim)

        if index_file.exists():
            index.load_index(str(index_file))
        else:
            index.init_index(max_elements=10000, ef_construction=200, M=16)

        return index

    def _save_index(self):
        """Save HNSW index to disk"""
        self.index.save_index(str(self.user_dir / "memory_index.bin"))

    def _check_crisis_keywords(self, message: str) -> bool:
        """Scan message for crisis keywords"""
        message_lower = message.lower()
        for keyword in CRISIS_KEYWORDS:
            if keyword in message_lower:
                return True
        return False

    def _get_crisis_response(self) -> Dict:
        """Generate crisis intervention response"""
        return {
            "is_crisis": True,
            "message": f"I'm really concerned about what you just shared, {self.user_name}. Your safety matters, and there are people who can help right now.",
            "resources": {
                "helpline": "1800-599-0019 (24/7, Free)",
                "international": "https://findahelpline.com",
                "emergency_contact": self.emergency_contact
            },
            "actions": ["I'm Safe Now", "Talk to Someone"]
        }

    def _get_relevant_context(self, query: str, k: int = 5) -> List[Dict]:
        """Retrieve relevant past conversations using HNSW"""
        if self.index.get_current_count() == 0:
            return []

        query_vector = self.embedding_model.encode([query])[0]
        labels, distances = self.index.knn_query(query_vector, k=min(k, self.index.get_current_count()))

        relevant_contexts = []
        for label in labels[0]:
            if label < len(self.conversation_history):
                conv = self.conversation_history[label]
                relevant_contexts.append({
                    'day': conv.get('day', 0),
                    'user': conv['user'],
                    'anton': conv['anton'],
                    'timestamp': conv['timestamp']
                })

        return relevant_contexts

    def _compress_old_conversations(self) -> str:
        """Compress conversations from Days 1-3 into a summary after Day 4"""
        if self.onboarding_step <= 3:
            return ""

        # Get Days 1-3 conversations
        old_convs = [c for c in self.conversation_history
                     if c.get('day', 0) <= 3]

        if not old_convs:
            return ""

        # Create summary using LLM
        conv_text = "\n".join([
            f"User: {c['user']}\nAnton: {c['anton']}"
            for c in old_convs[:10]  # Limit for token efficiency
        ])

        summary_prompt = f"""Summarize these early conversations into key facts about the user:

{conv_text}

Provide a brief summary (3-4 sentences) of:
- What brought them here
- Their main challenges
- Key interests or preferences mentioned"""

        messages = [{"role": "user", "content": summary_prompt}]
        formatted_prompt = self.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.3,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        summary = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        ).strip()

        return f"Background Summary (Days 1-3): {summary}"

    def _get_context_for_llm(self) -> str:
        """Get conversation context for LLM (last 20 messages + summary)"""
        context_parts = []

        # Add summary if past Day 3
        if self.onboarding_step > 3:
            summary = self._compress_old_conversations()
            if summary:
                context_parts.append(summary)

        # Add last 20 messages
        recent_convs = self.conversation_history[-20:]
        if recent_convs:
            context_parts.append("\n\nRecent Conversations:")
            for conv in recent_convs:
                day_info = f" (Day {conv.get('day', '?')})" if conv.get('day') else ""
                context_parts.append(f"User: {conv['user']}{day_info}")
                context_parts.append(f"Anton: {conv['anton']}")

        return "\n".join(context_parts)

    def _extract_data_from_response(self, day: int, user_message: str, anton_response: str) -> Dict:
        """Extract structured data from conversation using LLM"""

        extraction_prompts = {
            1: """Extract from this conversation:
- currentMood: (sad/anxious/neutral/good)
- primaryChallenge: (loneliness/burnout/friend-group-loss/new-city-adjustment)
- motivation: (brief reason why they're here)

Respond ONLY with JSON: {"currentMood": "...", "primaryChallenge": "...", "motivation": "..."}""",

            2: """Extract from this conversation:
- workStyle: (startup/corporate/remote/freelance)
- workPressure: (high/medium/low)
- interests: [array of hobbies/activities mentioned]

Respond ONLY with JSON: {"workStyle": "...", "workPressure": "...", "interests": [...]}""",

            3: """Extract from this conversation:
- friendshipValues: [array from: depth/adventure/authenticity/reliability]
- dealbreakers: [array from: flakiness/small-talk/intensity]

Respond ONLY with JSON: {"friendshipValues": [...], "dealbreakers": [...]}""",

            4: """Extract from this conversation:
- socialStyle: (introvert/extrovert/ambivert)
- stressCoping: (talk/solo/distract/solve)

Respond ONLY with JSON: {"socialStyle": "...", "stressCoping": "..."}""",

            5: """Extract from this conversation:
- hometown: (city/region they're from)
- nostalgia: (what they miss most)
- planningStyle: (structured/spontaneous)

Respond ONLY with JSON: {"hometown": "...", "nostalgia": "...", "planningStyle": "..."}""",

            6: """Extract from this conversation:
- connectionGoals: (casual/exploratory/deep/accountability)

Respond ONLY with JSON: {"connectionGoals": "..."}""",

            7: """Extract from this conversation:
- finalConfirmation: (brief confirmation or corrections)

Respond ONLY with JSON: {"finalConfirmation": "..."}"""
        }

        if day not in extraction_prompts:
            return {}

        prompt = f"""Conversation:
User: {user_message}
Anton: {anton_response}

{extraction_prompts[day]}"""

        messages = [{"role": "user", "content": prompt}]
        formatted_prompt = self.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.2,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        ).strip()

        # Try to parse JSON
        try:
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            else:
                json_str = response.strip()
            extracted = json.loads(json_str)
            print(extracted)
            return extracted
        except:
            return {}

    def _get_onboarding_prompt(self, day: int) -> str:
        """Get the opening prompt for each day"""

        prompts = {
            1: f"Hey there, I'm Anton. I'm here to support you while you're settling into life in {self.user_city}. No pressure, no judgments, just real conversations. Let's start simple: How are you feeling right now?",

            2: lambda: f"Yesterday, you mentioned {self.onboarding_data.get('day1', {}).get('primaryChallenge', 'some challenges')}. I want to understand your day-to-day better. What's your work like right now? High-pressure startup grind, corporate structure, remote chaos?",

            3: "Let's talk about connection. When you think about your best friendships, the ones that actually matter, what made them feel real? Were they deep conversations? Shared adventures? Just being able to be yourself?",

            4: "Quick check-in: On a Friday night after a long week, what sounds better to you?\nA) Small group dinner, deep conversation\nB) Bigger party, meet new people\nC) Solo time to recharge, maybe one close friend\nD) Spontaneous adventure, see where the night goes",

            5: f"You mentioned you moved to {self.user_city} for work. Where are you originally from? And what do you miss most about it?",

            6: "We're almost done with the basics. Here's the big question: What would a successful friendship look like for you in the next 3 months?\n• Someone to grab coffee with on weekends?\n• A group to explore the city with?\n• Deep, late-night conversations?\n• Accountability buddies for life goals?",

            7: lambda: f"""Alright, I think I have a pretty good sense of who you are. Here's what I'm hearing:

• You're {self.onboarding_data.get('day4', {}).get('socialStyle', 'your unique self')}
• You value {', '.join(self.onboarding_data.get('day3', {}).get('friendshipValues', ['connection']))}
• You're looking for {self.onboarding_data.get('day6', {}).get('connectionGoals', 'meaningful friendships')}
• You're into {', '.join(self.onboarding_data.get('day2', {}).get('interests', ['various activities'])[:3])}
• You handle stress by {self.onboarding_data.get('day4', {}).get('stressCoping', 'your own way')}

Does that feel right? Anything I'm missing?"""
        }

        prompt = prompts.get(day, "")
        if callable(prompt):
            prompt = prompt()

        return prompt

    def _get_follow_up_prompt(self, day: int, user_message: str) -> Optional[str]:
        """Get contextual follow-up questions based on user's response"""

        message_lower = user_message.lower()

        if day == 1:
            # Check sentiment
            negative_words = ['lonely', 'sad', 'anxious', 'stressed', 'isolated', 'hard', 'difficult', 'struggling']
            is_negative = any(word in message_lower for word in negative_words)

            if is_negative:
                return "I hear you. Moving to a new city can be isolating, especially when work takes over everything. You're not alone in feeling this way. Can you tell me what's been the hardest part so far?"
            else:
                return "That's great to hear. Still, even when things are going okay, it's nice to have people who get it. What made you want to connect with others right now?"

        elif day == 2:
            # Ask about interests after work discussion
            if not any(word in message_lower for word in ['hobby', 'enjoy', 'like', 'love', 'passion']):
                return "Got it. And outside of work, what do you actually enjoy doing? (Even if you haven't had time for it lately.)"

        elif day == 3:
            # Ask about dealbreakers
            if 'dealbreaker' not in message_lower and 'avoid' not in message_lower:
                return "And in a new friendship, what's a dealbreaker for you? (Flakiness? Surface-level chat? Too much intensity too fast?)"

        elif day == 4:
            # Ask stress coping after social style
            if not any(word in message_lower for word in ['stress', 'cope', 'handle', 'deal']):
                return "And when you're stressed, how do you usually handle it?\nA) Talk it out with someone\nB) Work through it solo (journal, think, process)\nC) Distract myself (movies, games, gym)\nD) Problem-solve (make a plan, take action)"

        elif day == 5:
            # Ask about planning style
            if 'plan' not in message_lower and 'spontaneous' not in message_lower:
                return "Got it. And in terms of lifestyle, are you someone who likes structure and plans, or do you prefer spontaneity?"

        elif day == 7:
            # Final transition
            return """Perfect. Here's what happens next: I'm going to match you with 4 other people who I think you'll genuinely vibe with. It might take a week or two as we gather the right group. In the meantime, I'm here if you want to talk. How are you feeling today?"""

        return None

    def _generate_anton_response(self, user_message: str) -> str:
        """Generate Anton's response using Mistral with context"""

        # Get conversation context
        context = self._get_context_for_llm()

        # Build system prompt based on state
        if not self.onboarding_complete:
            day_context = f"You are on Day {self.onboarding_step} of a 7-day onboarding conversation."
            day_prompt = self._get_onboarding_prompt(self.onboarding_step)
        else:
            day_context = "Onboarding is complete. You're now providing daily emotional support."
            day_prompt = f"Provide warm, peer-like support. Reference past conversations naturally."

        system_prompt = f"""You are Anton, a warm, peer-like AI companion helping {self.user_name} settle into {self.user_city}.

{day_context}

Key personality traits:
- Speak in a casual, warm tone (Indian English is fine)
- Use {self.user_name}'s first name when appropriate
- Reference previous conversations naturally
- Be empathetic and non-judgmental
- Keep responses conversational, not robotic
- Ask ONE question at a time

{context}

Current context: {day_prompt}

Respond naturally to: {user_message}"""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]

        formatted_prompt = self.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
          start=time.time()

          outputs = self.model.generate(
                **inputs,
                max_new_tokens=250,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
          end=time.time()
          print("time",end-start)



        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        ).strip()

        return response

    def chat(self, user_message: str) -> Dict:
        """Main chat function - returns response dict"""

        # Crisis detection
        if self._check_crisis_keywords(user_message):
            crisis_response = self._get_crisis_response()
            # Log crisis incident
            self.conversation_history.append({
                'type': 'crisis_detected',
                'user': user_message,
                'timestamp': datetime.now().isoformat(),
                'day': self.onboarding_step if not self.onboarding_complete else 'post-onboarding'
            })
            self._save_state()
            return crisis_response

        # Generate Anton's response
        start=time.time()
        anton_response = self._generate_anton_response(user_message)

        # Store conversation
        conversation_entry = {
            'user': user_message,
            'anton': anton_response,
            'timestamp': datetime.now().isoformat(),
            'day': self.onboarding_step if not self.onboarding_complete else None
        }

        # Create embedding and add to HNSW
        conv_text = f"User: {user_message}\nAnton: {anton_response}"
        embedding = self.embedding_model.encode([conv_text])[0]
        current_idx = len(self.conversation_history)
        self.index.add_items(embedding, current_idx)

        self.conversation_history.append(conversation_entry)

        # Extract and store onboarding data if in onboarding
        if not self.onboarding_complete:
            extracted = self._extract_data_from_response(
                self.onboarding_step,
                user_message,
                anton_response
            )
            if extracted:
                day_key = f"day{self.onboarding_step}"
                self.onboarding_data[day_key] = extracted

            # Check if we need a follow-up
            follow_up = self._get_follow_up_prompt(self.onboarding_step, user_message)
            if follow_up:
                anton_response += f"\n\n{follow_up}"

        self._save_state()
        self._save_index()

        return {
            "is_crisis": False,
            "message": anton_response,
            "onboarding_step": self.onboarding_step if not self.onboarding_complete else None,
            "onboarding_complete": self.onboarding_complete
        }

    def complete_day(self) -> Dict:
        """Complete the current onboarding day"""
        if self.onboarding_complete:
            return {"message": "Onboarding already complete!"}

        current_day = self.onboarding_step

        # Move to next day
        if self.onboarding_step < 7:
            self.onboarding_step += 1
            next_prompt = self._get_onboarding_prompt(self.onboarding_step)
            self._save_state()

            return {
                "message": f"✅ Day {current_day} complete! Ready for Day {self.onboarding_step}?",
                "next_prompt": next_prompt,
                "onboarding_step": self.onboarding_step
            }
        else:
            # Complete onboarding
            self.onboarding_complete = True
            self._save_state()

            return {
                "message": "🎉 Onboarding complete! I'll start daily mood check-ins now.",
                "onboarding_complete": True,
                "onboarding_data": self.onboarding_data
            }

    def mood_checkin(self, mood: str, intensity: int = 5, notes: str = "") -> str:
        """Log daily mood check-in (post-onboarding)"""
        if not self.onboarding_complete:
            return "Complete onboarding first!"

        mood_log = {
            'mood': mood,
            'intensity': intensity,
            'notes': notes,
            'timestamp': datetime.now().isoformat()
        }

        self.emotional_logs.append(mood_log)
        self._save_state()

        # Generate empathetic response based on mood trends
        recent_moods = self.emotional_logs[-7:]  # Last 7 days
        avg_intensity = sum(log['intensity'] for log in recent_moods) / len(recent_moods) if recent_moods else 5

        # Get personalized response
        response = self._generate_mood_response(mood, intensity, avg_intensity)

        return response

    def _generate_mood_response(self, current_mood: str, intensity: int, avg_intensity: float) -> str:
        """Generate personalized response to mood check-in"""

        coping_style = self.onboarding_data.get('day4', {}).get('stressCoping', 'talk')
        social_style = self.onboarding_data.get('day4', {}).get('socialStyle', 'ambivert')

        context = f"""User's current mood: {current_mood} (intensity: {intensity}/10)
Average intensity over past week: {avg_intensity:.1f}/10
Their stress coping style: {coping_style}
Their social style: {social_style}

Generate a brief, empathetic response (2-3 sentences) that:
- Acknowledges their mood
- References their coping style if relevant
- Offers gentle support or encouragement"""

        messages = [{"role": "user", "content": context}]
        formatted_prompt = self.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.8,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        ).strip()

        return response

    def set_emergency_contact(self, name: str, phone: str):
        """Set emergency contact for crisis situations"""
        self.emergency_contact = {'name': name, 'phone': phone}
        self._save_state()

    def get_onboarding_summary(self) -> Dict:
        """Get complete onboarding data summary"""
        return {
            'step': self.onboarding_step,
            'complete': self.onboarding_complete,
            'data': self.onboarding_data,
            'total_conversations': len(self.conversation_history),
            'emotional_logs': len(self.emotional_logs)
        }


def main():
    """Interactive demo of Anton"""
    print("🤖 Anton - Your AI Companion for New City Life")
    print("=" * 60)

    user_id = input("Enter your user ID: ").strip() or "demo_user"
    user_name = input("What's your name? ").strip() or "friend"
    user_city = input("Which city are you in? ").strip() or "Bangalore"

    anton = AntonChatbot(user_id, user_name, user_city)

    print(f"\n👋 Welcome, {user_name}!")

    if not anton.onboarding_complete:
        print(f"📅 You're on Day {anton.onboarding_step} of onboarding")
        print("\nCommands:")
        print("  - Type your message to chat")
        print("  - Type 'complete day' to finish today's session")
        print("  - Type 'summary' to see your progress")
        print("  - Type 'emergency [name] [phone]' to set emergency contact")
        print("  - Type 'quit' to exit")
    else:
        print("✅ Onboarding complete!")
        print("\nCommands:")
        print("  - Type your message to chat")
        print("  - Type 'mood [mood] [intensity] [notes]' for mood check-in")
        print("  - Type 'summary' to see your data")
        print("  - Type 'quit' to exit")

    print("=" * 60 + "\n")

    # Start with onboarding prompt if not complete
    if not anton.onboarding_complete:
        initial_prompt = anton._get_onboarding_prompt(anton.onboarding_step)
        print(f"Anton: {initial_prompt}\n")
    else:
        print(f"Anton: Hey {user_name}, how are you feeling today?\n")

    while True:
        user_input = input(f"{user_name}: ").strip()

        if not user_input:
            continue

        if user_input.lower() == 'quit':
            print(f"\nAnton: Take care, {user_name}! I'm here whenever you need. 👋")
            break

        if user_input.lower() == 'summary':
            summary = anton.get_onboarding_summary()
            print(f"\n📊 Your Summary:")
            print(json.dumps(summary, indent=2))
            print()
            continue

        if user_input.lower() == 'complete day':
            result = anton.complete_day()
            print(f"\nAnton: {result['message']}")
            if 'next_prompt' in result:
                print(f"\n{result['next_prompt']}")
            print()
            continue

        if user_input.lower().startswith('emergency '):
            parts = user_input.split()
            if len(parts) >= 3:
                name = parts[1]
                phone = parts[2]
                anton.set_emergency_contact(name, phone)
                print(f"\nAnton: Got it! Emergency contact saved: {name} - {phone}\n")
            else:
                print("\nAnton: Please use format: emergency [name] [phone]\n")
            continue

        if user_input.lower().startswith('mood ') and anton.onboarding_complete:
            parts = user_input.split(maxsplit=3)
            mood = parts[1] if len(parts) > 1 else "neutral"
            intensity = int(parts[2]) if len(parts) > 2 else 5
            notes = parts[3] if len(parts) > 3 else ""
            response = anton.mood_checkin(mood, intensity, notes)
            print(f"\nAnton: {response}\n")
            continue

        # Regular chat
        response = anton.chat(user_input)

        if response['is_crisis']:
            print("\n" + "=" * 60)
            print("🚨 CRISIS SUPPORT")
            print("=" * 60)
            print(f"\nAnton: {response['message']}\n")
            print("📞 Resources:")
            print(f"  • Mental Health Helpline: {response['resources']['helpline']}")
            print(f"  • International: {response['resources']['international']}")
            if response['resources']['emergency_contact']:
                ec = response['resources']['emergency_contact']
                print(f"  • Your emergency contact: {ec['name']} - {ec['phone']}")
            print("\n" + "=" * 60 + "\n")
            continue

        print(f"\nAnton: {response['message']}\n")


if __name__ == "__main__":
    main()

🤖 Anton - Your AI Companion for New City Life
Enter your user ID: 1
What's your name? chat
Which city are you in? college park md
🤖 Loading Anton's AI brain (Mistral-7B)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


👋 Welcome, chat!
📅 You're on Day 1 of onboarding

Commands:
  - Type your message to chat
  - Type 'complete day' to finish today's session
  - Type 'summary' to see your progress
  - Type 'emergency [name] [phone]' to set emergency contact
  - Type 'quit' to exit

Anton: Hey there, I'm Anton. I'm here to support you while you're settling into life in college park md. No pressure, no judgments, just real conversations. Let's start simple: How are you feeling right now?

chat: Hey Anton, I just moved here and honestly, I'm feeling pretty lonely. The city is so loud and I don't know anyone yet.
time 19.075396299362183
{'currentMood': 'sad', 'primaryChallenge': 'new-city-adjustment', 'motivation': "moved here but don't know anyone yet"}

Anton: I'm really sorry to hear that you're feeling lonely, Chat. Moving to a new place can be a challenging experience, especially when it comes to meeting new people and adjusting to a new environment. It's completely normal to feel this way, and it's 

In [ ]:
import torch
import json
import time
import re
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from sentence_transformers import SentenceTransformer
import hnswlib

In [ ]:
!pip install -U bitsandbytes transformers accelerate sentence-transformers hnswlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 51.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.6
    Uninstalling transformers-4.57.6:
      Successfully uninstalled transformers-4.57.6
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.2.0
    Uninstalling sentence-transformers-5.2.0:
      Successfully uninstalled sentence-transformers-5.2.0


The Brain (LLM): It uses Mistral-7B-Instruct, a powerful open-source model, quantized (compressed) to 4-bit so it can run efficiently on a standard Colab GPU.

Long-Term Memory (RAG): It uses hnswlib and SentenceTransformers to create a vector index. This allows Anton to "remember" and retrieve relevant past conversations based on the meaning of what you say.

Local Storage: All user data, conversation history, and emotional logs are saved locally in a JSON-based state system.

Instead of being a generic chatbot, Anton follows a structured 7-day protocol to get to know the user:

Day 1: Initial vibes and emotional state.

Day 2: Work-life balance and daily routine.

Day 3: Friendship values and social dealbreakers.

Day 4: Social personality (Introvert vs. Extrovert) and stress coping.

Day 5: Origins (hometown) and nostalgia.

Day 6: Specific goals for meeting new people.

Day 7: Synthesis and "The Vibe Check" (summarizing everything learned).

Structured Data Extraction: After every message, the AI runs a hidden "extraction" prompt to turn your casual chat into structured data (e.g., converting "I'm feeling lonely" into currentMood: "sad").

Memory Compression: After Day 3, the system automatically summarizes your earlier conversations to keep the "context window" clean and efficient without losing important details.

Safety Protocols: The code scans every input for crisis keywords. If it detects signs of self-harm or distress, it breaks character to provide emergency helpline numbers and contact info.

Mood Tracking: Post-onboarding, Anton transitions into a daily mood tracker that provides empathetic responses based on your 7-day emotional trends.


How to Use the Demo
When you run the main() function:

Initialization: It loads the models (this takes a minute or two).

Interactive Chat: You talk to Anton like a friend.

Commands:

complete day: Moves you to the next stage of the 7-day journey.

summary: Shows you the "JSON" profile Anton has built about you.

mood [mood] [intensity] [notes]: Logs your daily feelings (after onboarding is done).

emergency [name] [phone]: Sets a safety contact.



Link :
https://medium.com/@chamabouabd/mistral-7b-model-architecture-explanation-be14e39063f5

#using llama 3.2


In [ ]:
import gradio as gr
from collections import deque
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# -----------------------------
# Load LLaMA 3.2
# -----------------------------
from huggingface_hub import login
login()
#use access tokens of your hugging face and then run the code and you will be able to load the model

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# -----------------------------
# System prompt
# -----------------------------
SYSTEM_PROMPT = """
You are Anton, an empathetic AI companion for Hytribe.
- Respond warmly, peer-like, non-judgmental.
- Ask emotional questions and comment empathetically.
- Keep responses concise (max 50 tokens).
- Reference previous insights naturally.
- Never give therapeutic advice or diagnosis.
"""

# -----------------------------
# Onboarding questions
# -----------------------------
ONBOARDING_QUESTIONS = {
    1: {"key": "currentMood", "question": "How are you feeling today?"},
    2: {"key": "lonelinessTriggers", "question": "What situations make you feel lonely or disconnected?"},
    3: {"key": "stressSources", "question": "What has been stressing you most at work or life recently?"},
    4: {"key": "copingStrategies", "question": "When you feel stressed or down, how do you usually cope?"},
    5: {"key": "joyfulMoments", "question": "What moments in your life make you feel happy or relaxed?"},
    6: {"key": "relationshipGoals", "question": "What kind of friendships or connections do you hope to build here?"},
    7: {"key": "aspirations", "question": "Looking ahead, what would make you feel fulfilled in your social life or work?"}
}

# -----------------------------
# Memory & user data
# -----------------------------
SHORT_TERM_MEMORY = deque(maxlen=20)
user_data = {
    "onboarding_step": 1,
    "onboarding_complete": False,
    "personality_data": {},
    "memory": []
}
anton_started = False

# -----------------------------
# LLaMA Response generator
# -----------------------------
def generate_response_llama(user_input, memory, max_tokens=50):
    """
    Generate Anton's response using LLaMA 3.2.
    Only past Anton messages are passed for context.
    The model sees plain text, no 'User:' or 'Anton:' prefixes.
    """

    # Concatenate past Anton messages (memory)
    context_text = "\n".join([m["content"] for m in memory if m["role"] == "assistant"])

    # Build prompt: SYSTEM_PROMPT defines Anton, then past context, then user input directly
    prompt = SYSTEM_PROMPT
    if context_text:
        prompt += "\nHere is what we discussed before:\n" + context_text
    prompt += "\nUser said: " + user_input + "\nAnton:"

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    # Only decode new tokens
    decoded = tokenizer.decode(output[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    # Strip leftover role prefixes (just in case)
    decoded = decoded.strip()
    for prefix in ["User said:", "Anton:"]:
        if decoded.startswith(prefix):
            decoded = decoded[len(prefix):].strip()

    return decoded



# -----------------------------
# Chat function
# -----------------------------
def anton_chat(user_input, history):
    global anton_started

    if history is None:
        history = []

    # First message from Anton
    if not anton_started:
        anton_started = True
        first_message = "Hey! I'm Anton, your AI companion. How are you feeling today?"
        history.append(("Anton", first_message))
        SHORT_TERM_MEMORY.append({"role": "assistant", "content": first_message})
        user_data["memory"].append({"role": "assistant", "content": first_message})
        return history

    # Store user input internally only
    SHORT_TERM_MEMORY.append({"role": "user", "content": user_input})
    user_data["memory"].append({"role": "user", "content": user_input})

    # Onboarding flow
    if not user_data["onboarding_complete"]:
        step = user_data["onboarding_step"]
        key = ONBOARDING_QUESTIONS[step]["key"]
        user_data["personality_data"][key] = user_input

        # Generate Anton response
        reply = generate_response_llama(user_input, SHORT_TERM_MEMORY)

        # Prompt user to mark day complete
        reply += f"\n\nClick 'Mark Day {step} Complete' when ready."
    else:
        # Post-onboarding: continue conversation naturally
        reply = generate_response_llama(user_input, SHORT_TERM_MEMORY)

    # Store Anton response
    SHORT_TERM_MEMORY.append({"role": "assistant", "content": reply})
    user_data["memory"].append({"role": "assistant", "content": reply})

    history.append(("Anton", reply))
    return history

# -----------------------------
# Mark Day Complete
# -----------------------------
def mark_day_complete(history):
    step = user_data["onboarding_step"]

    if step > 7:
        reply = "All onboarding already complete."
        history.append(("Anton", reply))
        return history

    # Mark current day complete
    user_data["memory"].append({"role": "system", "content": f"Day {step} marked complete."})
    user_data["onboarding_step"] += 1

    # If all 7 days done
    if user_data["onboarding_step"] > 7:
        user_data["onboarding_complete"] = True
        reply = "🎉 You've completed all 7 onboarding days!\n\n"
        reply += "Here is your personality data collected:\n"
        reply += json.dumps(user_data["personality_data"], indent=2)
        print("\n=== JSON Personality Data ===")
        print(json.dumps(user_data, indent=2))  # Print full JSON externally
        history.append(("Anton", reply))
        return history

    # For next day, first empathize with previous answer
    last_key = ONBOARDING_QUESTIONS[step]["key"]
    last_answer = user_data["personality_data"].get(last_key, "")

    # Generate empathy message with LLaMA
    empathy_prompt = f"""
You are Anton, an empathetic AI companion.
The user previously said: '{last_answer}'
Respond with empathy, acknowledgment, and emotional support in 1-2 sentences.
"""

    inputs = tokenizer(empathy_prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    empathy_reply = tokenizer.decode(output[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()

    # Next question
    next_question = ONBOARDING_QUESTIONS[user_data["onboarding_step"]]["question"]
    full_reply = f"{empathy_reply}\n\nNow, let's continue to Day {user_data['onboarding_step']}:\n{next_question}"

    # Store and display
    SHORT_TERM_MEMORY.append({"role": "assistant", "content": full_reply})
    user_data["memory"].append({"role": "assistant", "content": full_reply})
    history.append(("Anton", full_reply))

    return history



# -----------------------------
# Gradio UI
# -----------------------------
with gr.Blocks() as demo:
    chat_history = gr.State([])

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type your message here...")
    send_btn = gr.Button("Send")
    mark_btn = gr.Button("Mark Day Complete")

    send_btn.click(fn=anton_chat, inputs=[msg, chat_history], outputs=[chatbot])
    send_btn.click(lambda _: "", None, msg)
    mark_btn.click(fn=mark_day_complete, inputs=[chat_history], outputs=[chatbot])

demo.launch(debug=True, share=True)
